In [ ]:
%load_ext autoreload
%autoreload 2

# Sonecule: ContinuousPMS – Continuous Parameter-Mapping Sonification

This notebook introduces and demonstrates usage of the ContinousPMS sonecule.
* The sonecule uses a synth that creates a continuous sound stream 
* for that a synth is used that offers several parameters that can be modulated
* Specifically the parameters are:
  * amplitude
  * frequency
  * number of harmonics
  * spatial panning
* Most likely, a custom synth will be created and passed on for individual sonifications, replacing the default.
* The mapping specifies how data channels shall control the individual parameters
* 

In [ ]:
# headers and imports for the demo
import sonecules as sn
import sc3nb as scn
from pya import Asig
import pyamapping as pam
import matplotlib.pyplot as plt
import time

# setup for matplotlib 
plt.rcParams["figure.figsize"] = (8,3)
%matplotlib widget

def a2d(**kwargs):
    return kwargs 

# start sonecules (with default backend sc3nb, aka sc3)
sn.startup()
ctx = sn.gcc()  # get the context as ctx

Load data sets used for the demo

In [ ]:
%run ../examples/prepare-data.ipynb

In [ ]:
# # select test data for sonecule, here EEG data from an epilepsy
# dasig = Asig(eeg_data, sr=250)
# plt.figure(figsize=(12,2)); 
# plt.subplot(121); dasig.plot(offset=1)

# data = dasig[{7:11}, [0,1,2,5,9,12]][::5]
# plt.subplot(122); data.plot(offset=2)

In [ ]:
df = bld_df
df['nr'] = df.index
df.iloc[:,10:-1].plot(lw=0.5, subplots=True, figsize=(9,6)); plt.tight_layout()
df.head()

## Usage Demo for the ContinuousPMS Sonecule

In [ ]:
from mesonic.synth import Synth
from sonecules.scoresyn import ContinuousPMS

The following code cell shows everything needed 
- to create the sonecule with data, 
- to clear the auditory canvas (aka timeline)
- to start the playback at a given rate
- to plot the timeline.

Usually we want to use a specific synths which we would define using the backend.
- In this case the default backend is sc3nb, so we can create it using scn.SynthDef
- Later sonecules will offer a well curated library of pre-packaged synths, so that it is rarely necessary to craft your own.
- The default synth (if none is provided is "cpmssyn", a 'continuous synth for PMSon'). 
- It is a pitched tone with added vibrato 
- It offers the continuous controls:

| parameter   | range  | meaning                                         |
| ----------- | ------ | ----------------------------------------------- |
| freq:       | [20..] | frequency                                       |
| amp         | [0,1]  | amplitude                                       |
| brightness: | [0,10] | brightness, the higher the more sharp the sound |
| pan         | [-1,1] | spatial panning from left to right              |
| plfreq      | [0..]  | multiplied pulse frequency                      |
| plwid       | [0,1]  | duty cycle of the pulse                         |
| plint       | [0,1]  | intensity of the pulse modulation               |

As currently no default synths are implemented we have to do the work ourselves

In [ ]:
scn.SynthDef("syc0", 
"""{ | out=0, freq=400, amp=0.1, plfreq=0, plwid=0.5, plint=0, brightness=0, pan=0, lg=0 | 
    var f = freq.lag(lg);
    var pulse = LFPulse.ar(plfreq, width: plwid, mul: plint, add: 1-plint);
    var tone = HPF.ar(Formant.ar(f, f, bwfreq: f * (brightness + 1)), 40);
    Out.ar(out, Pan2.ar(tone*pulse, pan.lag(lg), level: amp.lag(lg)));
}""").add()

scn.SynthDef("sycvib", 
"""{ | out=0, freq=400, amp=0.1, vibfreq=0, vibintrel=0, brightness=0, pan=0, lg=0 | 
    var vib = SinOsc.ar(vibfreq, mul: vibintrel*freq, add: freq.lag(lg));
    var sig = HPF.ar(Formant.ar(vib, vib, bwfreq: vib*brightness+1, mul: amp.lag(lg)), 40);
    Out.ar(out, Pan2.ar(sig, pan.lag(lg)));
}""").add()

scn.SynthDef("sycnoise", 
"""{ | out=0, freq=400, amp=0.1, rq=0.1, pan=0, lg=0 | 
    var sig = BPF.ar(WhiteNoise.ar(amp), freq.lag(lg), rq.lag(lg));
    Out.ar(out, Pan2.ar(sig, pan.lag(lg)));
}""").add()

scn.SynthDef("syctick", 
"""{ | out=0, freq=400, cf=4000, amp=0.1, pan=0, lg=0 | 
    var sig = LPF.ar(Impulse.ar(freq.lag(lg)), cf.lag(lg));
    Out.ar(out, Pan2.ar(sig, pan.lag(lg), amp));
}""").add()


In [ ]:
# # demo for the syctick
# sx = scn.Synth("syctick", a2d(freq=80, amp=0.05, cf=2000, pan=0))
# time.sleep(1); sx.set(a2d(lg=0.5, freq=15, cf=3000, amp=0.2, pan=-1))
# time.sleep(1); sx.set(a2d(lg=0.5, freq=5, cf=7000, amp=0.1, pan=1))
# time.sleep(1); sx.set(a2d(lg=0.5, freq=25, cf=2000, amp=0.1, pan=1)) 
# time.sleep(1); sx.set(a2d(lg=0.5, freq=45, cf=2000, amp=0.3, pan=1)) 
# time.sleep(1); sx.free()
# # demo for the sycnoise
# sx = scn.Synth("sycnoise", a2d(freq=800, amp=0.05, rq=0.1, pan=0))
# time.sleep(1); sx.set(a2d(lg=0.5, freq=500,  rq=0.1, amp=0.2, pan=-1))
# time.sleep(1); sx.set(a2d(lg=0.5, freq=5000, rq=0.5, amp=0.1, pan=1))
# time.sleep(1); sx.set(a2d(lg=0.5, freq=2000, rq=1.2, amp=0.1, pan=1)) 
# time.sleep(1); sx.set(a2d(lg=0.5, freq=400,  rq=0.1, amp=0.3, pan=1)) 
# time.sleep(1); sx.free()
# use this code to explore the synths as you see fit
# def a2d(**kwargs):
#     return kwargs 
# sx = scn.Synth("sycpms", a2d(freq=200, amp=0.05, plfreq=5, plwid=0.1, brightness=4, pan=0))
# sx.pfreq = 20
# sx.plwid = 0.7
# sx.set('plint', 0.5)
# sx.free()
# scn.SC.default.server.free_all()

The next cell shows all steps in specifying the mapping in one go

In [ ]:
# create a sonecule, initialized with the synth to be used
# second argument can provide defaults and bounds for parameters
sncpms = ContinuousPMS("syc0", { 
    "freq": {"bounds": (100, 5000)},
    "amp": {"default": 0.1},
    "brightness": {"default": 0},
    "lg": {"default": 0.1}
    })

# the mapping is just a dictionary where keys are the parameters,
# and values are tuples of 
# - data column, 
# - mapping type, and  
# - arguments of the mapping function as dictionary, e.g.
test_mapping = {
    "onset": ("nr", "lin", {"y1": 0, "y2": 4}),
    "freq" : ("humidity", "exp", {"y1": pam.midi_to_cps(50), "y2": pam.midi_to_cps(70)}),
    "brightness" : ("temperature", "lin", {"y1": 0, "y2": 10}),
    "pan" : ("solar_radiation", "lin", {"y1": -1, "y2": 1}),
}

# clear the timeline 
ctx.timeline.reset() 

# and render the sonification into the timeline
sncpms.schedule(df=df.iloc[0:10*24], mapping=test_mapping)

# finally start the realtime playback at a given rate
sncpms.start(rate=1)

Usually this can be done in a more condensed way, by
- omitting the defaults and bounds (later synths will come with good such values anyway)
- using shortcuts such as providing [min, max] instead of {"y1": min, "y2": max}
- setting constant values by value
- defining the mapping in the call
- starting the sonification by daisy chaining
as shown here

In [ ]:
# create a sonecule, initialized with your data selection
sncpms = ContinuousPMS("syc0")
ctx.timeline.reset() 
sncpms.schedule(df=df.iloc[4*24:14*24], mapping={
    "onset":        ("nr", "lin", (0, 10)),
    "freq" :        ("humidity", "exp", (100, 400)),
    "brightness" :  ("temperature", "lin", (2, 8)),
    "pan" :         ("solar_radiation", "lin", (-1, 1)),
    "amp" :         ("wind_speed", "exp", (0.02, 1)),
    "lg" : 0.05,
    }).start()

# as bonus: lets plot the used data
df.iloc[4*24:14*24].loc[:, ['humidity', 'temperature', 'solar_radiation', 'wind_speed']].plot()

Note that the events remain in the timeline. 
Setting the 

In [ ]:
ctx.timeline

Setting the time actively to 0 will cue the playback to that onset and result in
a sonification to be replayed

In [ ]:
ctx.realtime_playback.time = 0

In [ ]:
# to free the timeline use
ctx.clear()

In [ ]:
# to stop all sound playing via the backend use 
ctx.stop()

In [ ]:
# note that the playback's latency is >0 - it can also be set
# but see mesonic for details and help
ctx.realtime_playback.processor.latency

In [ ]:
# for own mapping experiments, its useful to see all columns
df.columns

In [ ]:
# as long as we reuse the synth, no need to create the object, but just use with different mappings
ctx.timeline.reset() 
sncpms.schedule(df.iloc[10*24:60*24], mapping={
    "onset":        ("nr",                  "lin", [0, 10]),
    "freq" :        ("solar_radiation",     "exp", [pam.midi_to_cps(40), pam.midi_to_cps(52)]),
    "brightness" :  ("temperature",         "lin", [0, 10]),
    "pan" :         ("humidity",            "lin", [-1, 1]),
    "amp" : 0.2
}
).start(rate=1)

### Hyrbid Continuous Parameter Mappings

Let's now explore a more interesting mapping:
- together with the above mapping of s
- to play impulses for the electricity
- wind sounds for the wind speed
- and map the water use on pulse choppings of the stream

In [ ]:
ctx.managers['synths']

In [ ]:
# create a sonecule, initialized with your data selection
dfsel = df.iloc[4*24:14*24]
ctx.timeline.reset() 
onset_mapping_spec = ("nr", "lin", (0, 10))

s1 = ContinuousPMS("syc0").schedule(dfsel, {
    "onset":        onset_mapping_spec,
    "freq" :        ("temperature", "exp", (100, 400)),
    "brightness" :  ("humidity", "lin", (2, 8)),
    "pan" :         0,
    "amp" :         ("solar_radiation", "exp", (0.05, 0.5)),
    "lg" : 0.05,
    })

s2 = ContinuousPMS("sycnoise").schedule(dfsel, {
    "onset":        onset_mapping_spec,
    "freq" :        ("wind_speed", "exp", (100, 2000)),
    "pan" :         0.5,
    "rq" :          0.4,
    "amp" :         ("wind_speed", "exp", (0.2, 0.8)),
    "lg" : 0.05,
    })

s3 = ContinuousPMS("syctick").schedule(dfsel, {
    "onset":    onset_mapping_spec,
    "freq" :    ("hc_wb_electrical", "exp", (10, 80)),
    "pan" : -0.5, "cf": 8000, "amp":  1, "lg": 0.05,
    })

# as bonus: lets plot the used data
dfsel.iloc[:,10:-2].plot(subplots=True, figsize=(10,6));

ctx.realtime_playback.start()

Using the a2d, this could also be written a bit more like code, as

In [ ]:
# create a sonecule, initialized with your data selection
dfsel = df.iloc[4*24:14*24]
ctx.timeline.reset() 

onset_mapping_spec = ("nr", "lin", (0, 10))

s1 = ContinuousPMS("syc0").schedule(dfsel, a2d(
    onset = onset_mapping_spec,
    freq = ("temperature", "exp", (100, 400)),
    brightness = ("humidity", "lin", (2, 8)),
    amp = ("solar_radiation", "exp", (0.05, 0.5)),
    pan = 0, lg = 0.05,
))

s2 = ContinuousPMS("sycnoise").schedule(dfsel, a2d(
    onset = onset_mapping_spec,
    freq = ("wind_speed", "exp", (100, 2000)),
    amp = ("wind_speed", "exp", (0.2, 0.8)),
    pan = 0.5, rq = 0.4, lg = 0.05,
))

s3 = ContinuousPMS("syctick").schedule(dfsel, a2d(
    onset = onset_mapping_spec,
    freq = ("hc_wb_electrical", "exp", (10, 80)),
    pan = -0.5, cf = 8000, amp = 1, lg = 0.05,
))

# as bonus: lets plot the used data
# dfsel.iloc[:,10:-2].plot(subplots=True, figsize=(10,6))

ctx.realtime_playback.start()

## Code Template

The following code snippets are intended for copy & paste to your notebooks, to facilitate getting your data sonified
using this sonecule.
* It is assumed that your data is stored in an Asig dasig

In [ ]:
# load your multi-channel data into an Asig, e.g. 
data = np.random.random((1000, 4))-0.5 # 100 rows with 8 channels, here same fake data
data = np.cumsum(data,axis=0)
df = pd.DataFrame(data, columns=["c1", "c2", "c3", "c4"])
df['nr'] = df.index
df.plot(subplots=True);

In [ ]:
scn.SynthDef("syc0", 
"""{ | out=0, freq=400, amp=0.1, plfreq=0, plwid=0.5, plint=0, brightness=0, pan=0, lg=0 | 
    var f = freq.lag(lg);
    var pulse = LFPulse.ar(plfreq, width: plwid, mul: plint, add: 1-plint);
    var tone = HPF.ar(Formant.ar(f, f, bwfreq: f * (brightness + 1)), 40);
    Out.ar(out, Pan2.ar(tone*pulse, pan.lag(lg), level: amp.lag(lg)));
}""").add()

# load your data / select your data
mydf = df

# sonecule for your synth with defaults and bounds
sncpms = ContinuousPMS("syc0", { 
    "freq": {"bounds": (100, 5000)},
    "amp": {"default": 0.5},
    "brightness": {"default": 0},
    "lg": {"default": 0.01}
    })

# mapping dictionary 
test_mapping = {
    "onset":       ("nr", "lin", [0, 8]),
    "freq" :       ("c1", "exp", {"y1": pam.midi_to_cps(50), "y2": pam.midi_to_cps(70)}),
    "brightness" : ("c2", "lin", {"y1": 0, "y2": 10}),
    "plfreq" :     ("c3", "lin", {"y1": 5, "y2": 25}),
    "plint" : 1, "plwid": 0.5,
}

# clear the timeline 
ctx.timeline.reset() 

# and render the sonification into the timeline
sncpms.schedule(df=mydf, mapping=test_mapping)

# finally start the realtime playback at a given rate
sncpms.start(rate=1)

# if needed: plot the timeline using 
ctx.timeline.plot()